In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, glob, shutil, json, random
BASE_DIR     = "/content/drive/MyDrive/nl2lp_project"
DATASET_DIR  = f"{BASE_DIR}/datasets"
MERGED_DIR   = f"{BASE_DIR}/merged"
CKPT_DIR     = f"{BASE_DIR}/checkpoints"
OUT_DIR      = f"{BASE_DIR}/nl2lp-phi3-qlora"   # LoRA adapter will be saved here

os.makedirs(DATASET_DIR, exist_ok=True)
os.makedirs(MERGED_DIR,  exist_ok=True)
os.makedirs(CKPT_DIR,    exist_ok=True)

# expected filenames in Drive
NL4_TRAIN = f"{DATASET_DIR}/train.jsonl"
NL4_VAL   = f"{DATASET_DIR}/val.jsonl"
NL4_NORM  = f"{DATASET_DIR}/nl4opt_normalized.jsonl"  # we will (re)build this
TSP_OUT   = f"{DATASET_DIR}/tsp_lp_dataset.jsonl"     # we will build this

print("DATASET_DIR:", DATASET_DIR)
print("MERGED_DIR:", MERGED_DIR)


Mounted at /content/drive
DATASET_DIR: /content/drive/MyDrive/nl2lp_project/datasets
MERGED_DIR: /content/drive/MyDrive/nl2lp_project/merged


In [ ]:
# normalize duplicate file names in datasets
candidates = glob.glob(f"{DATASET_DIR}/*.jsonl")
for p in candidates:
    base = os.path.basename(p)
    if base.startswith("train (") and base.endswith(").jsonl"):
        shutil.copy(p, NL4_TRAIN); print("copied ->", NL4_TRAIN)
    if base.startswith("val (") and base.endswith(").jsonl"):
        shutil.copy(p, NL4_VAL);   print("copied ->", NL4_VAL)

# sanity check
print("exists train:", os.path.exists(NL4_TRAIN), NL4_TRAIN)
print("exists val  :", os.path.exists(NL4_VAL),   NL4_VAL)


exists train: True /content/drive/MyDrive/nl2lp_project/datasets/train.jsonl
exists val  : True /content/drive/MyDrive/nl2lp_project/datasets/val.jsonl


In [ ]:
def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for ln in f:
            if ln.strip():
                rows.append(json.loads(ln))
    return rows

def save_jsonl(rows, path):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def norm_nl4(records):
    out = []
    for r in records:
        instr = (r.get("instruction") or
                 r.get("input") or
                 r.get("prompt") or
                 r.get("problem_text") or "").strip()
        ans   = (r.get("output") or
                 r.get("answer") or
                 r.get("lp_string") or "").strip()
        if instr and ans:
            out.append({"instruction": instr, "input": "", "output": ans, "source": "nl4opt"})
    return out

# must exist
assert os.path.exists(NL4_TRAIN), "train.jsonl missing"
assert os.path.exists(NL4_VAL),   "val.jsonl missing"

tr_raw = load_jsonl(NL4_TRAIN)
vl_raw = load_jsonl(NL4_VAL)
tr = norm_nl4(tr_raw)
vl = norm_nl4(vl_raw)

save_jsonl(tr+vl, NL4_NORM)
print(f"✅ Normalized NL4Opt -> {NL4_NORM} (train {len(tr)}, val {len(vl)})")


✅ Normalized NL4Opt -> /content/drive/MyDrive/nl2lp_project/datasets/nl4opt_normalized.jsonl (train 475, val 25)


In [ ]:
random.seed(42)

def gen_complete_graph(cities, asymmetric=False, lo=5, hi=60):
    C = {}
    n = len(cities)
    for i in range(n):
        for j in range(n):
            if i == j: continue
            if asymmetric:
                C[(cities[i], cities[j])] = random.randint(lo, hi)
            else:
                if j > i:
                    d = random.randint(lo, hi)
                    C[(cities[i], cities[j])] = d
                    C[(cities[j], cities[i])] = d
    return C

def mtz_text_formulation(cities, C):
    n = len(cities)
    # objective
    terms = [f"{c}*x_{i}_{j}" for (i,j), c in C.items()]
    objective = "Objective:\n  Minimize Z = " + " + ".join(terms)

    deg_out = [f"  sum_{{j in {cities}}} x_{i}_{{j}} = 1" for i in cities]
    deg_in  = [f"  sum_{{i in {cities}}} x_{{i}}_{j} = 1" for j in cities]

    mtz = [f"  u_{cities[0]} = 1", f"  1 <= u_i <= {n} for all i in {cities}"]
    for i in cities:
        for j in cities:
            if i != j:
                mtz.append(f"  u_{i} - u_{j} + {n}*x_{i}_{j} <= {n}-1")

    binvars = [f"  x_{i}_{j} in {{0,1}} for all i≠j in {cities}"]
    intvars = [f"  u_i integer for all i in {cities}"]

    constraints = ["Constraints:"] + deg_out + deg_in + mtz + binvars + intvars
    var_section = ("Decision Variables:\n"
                   "  x_{i,j} = 1 if traveling directly from i to j; 0 otherwise\n"
                   "  u_i are MTZ order variables (1..n)")

    uniq_pairs, seen = [], set()
    for (i,j), c in C.items():
        if (j,i) in seen or i == j: continue
        uniq_pairs.append(f"{i}-{j}={c}")
        seen.add((i,j)); seen.add((j,i))
    distances_nl = ", ".join(uniq_pairs)

    nl_prompt = (f"A salesman must visit all {n} cities exactly once and return. "
                 f"Cities: {', '.join(cities)}. Distances: {distances_nl}. "
                 f"Formulate an ILP using MTZ formulation.")
    lp_text = "\n".join([objective, "", *constraints, "", var_section])
    return nl_prompt, lp_text

N_SAMPLES = 600
with open(TSP_OUT, "w", encoding="utf-8") as f:
    for _ in range(N_SAMPLES):
        n = random.randint(4, 8)
        cities = [f"C{i+1}" for i in range(n)]
        C = gen_complete_graph(cities, asymmetric=(random.random() < 0.3))
        instr, out = mtz_text_formulation(cities, C)
        f.write(json.dumps({"instruction": instr, "input": "", "output": out, "source": "tsp"}) + "\n")
print(f"✅ Generated TSP -> {TSP_OUT} ({N_SAMPLES} rows)")


✅ Generated TSP -> /content/drive/MyDrive/nl2lp_project/datasets/tsp_lp_dataset.jsonl (600 rows)


In [ ]:
from sklearn.model_selection import train_test_split

def read_jsonl(path):
    return [json.loads(x) for x in open(path, "r", encoding="utf-8").read().splitlines() if x.strip()]

nl4_train = norm_nl4(read_jsonl(NL4_TRAIN))
nl4_val   = norm_nl4(read_jsonl(NL4_VAL))
tsp_all   = read_jsonl(TSP_OUT)

tsp_train, tsp_val = train_test_split(tsp_all, test_size=0.05, random_state=42)

merged_train = nl4_train + tsp_train
merged_val   = nl4_val   + tsp_val
random.shuffle(merged_train)
random.shuffle(merged_val)

MERGED_TRAIN = f"{MERGED_DIR}/train.jsonl"
MERGED_VAL   = f"{MERGED_DIR}/val.jsonl"
save_jsonl(merged_train, MERGED_TRAIN)
save_jsonl(merged_val,   MERGED_VAL)

print("✅ merged sizes:", len(merged_train), "(train),", len(merged_val), "(val)")


✅ merged sizes: 1045 (train), 55 (val)


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"   # 👈 switched model
MAX_LEN    = 1024

SYSTEM_PREFIX = (
    "You are an Operations Research expert. "
    "Given a natural-language problem, respond ONLY with a clean LP/ILP formulation:\n"
    "Objective:\nConstraints:\nDecision Variables:\n"
)

def build_text(rec):
    return f"{SYSTEM_PREFIX}\n\nProblem: {rec['instruction']}\nAnswer:\n{rec['output']}"

tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

def load_local_jsonl(path):
    return Dataset.from_list([json.loads(l) for l in open(path, "r").read().splitlines() if l.strip()])

train_ds = load_local_jsonl(MERGED_TRAIN)
val_ds   = load_local_jsonl(MERGED_VAL)

def map_format(ex):
    text = build_text(ex)
    enc = tok(text, max_length=MAX_LEN, truncation=True, padding="max_length")
    enc["labels"] = enc["input_ids"].copy()
    return enc

train_tok = train_ds.map(map_format, remove_columns=train_ds.column_names)
val_tok   = val_ds.map(map_format,   remove_columns=val_ds.column_names)

len(train_tok), len(val_tok)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1045 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

(1045, 55)

In [ ]:
!pip install bitsandbytes

In [ ]:
import os, torch
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

class DynamicPaddingCausalCollator:
    def __init__(self, tokenizer, pad_to_multiple_of=8):
        self.tokenizer = tokenizer
        self.pad_to_multiple_of = pad_to_multiple_of
    def __call__(self, features):
        batch = self.tokenizer.pad(features, padding=True, return_tensors="pt",
                                   pad_to_multiple_of=self.pad_to_multiple_of)
        labels = batch["input_ids"].clone()
        labels[batch["attention_mask"] == 0] = -100
        batch["labels"] = labels
        return batch

collator = DynamicPaddingCausalCollator(tok)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

print("Loading Phi-3 base model in 4-bit...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map={"": 0},
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)
model = get_peft_model(model, lora_cfg)

if hasattr(model, "enable_input_require_grads"):
    model.enable_input_require_grads()
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
model.config.use_cache = False

try:
    model.print_trainable_parameters()
except:
    print("Trainable tensors:", sum(p.requires_grad for p in model.parameters()))

args = TrainingArguments(
    output_dir=OUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=250,
    save_steps=250,
    save_total_limit=2,
    fp16=True,
    bf16=False,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    group_by_length=True,
    dataloader_num_workers=2,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=collator,
)

print("Starting training...")
trainer.train()


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 8,912,896 || all params: 3,829,992,448 || trainable%: 0.2327
Starting training...


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=131, training_loss=0.11210915787529399, metrics={'train_runtime': 2599.4137, 'train_samples_per_second': 0.402, 'train_steps_per_second': 0.05, 'total_flos': 2.39579687288832e+16, 'train_loss': 0.11210915787529399, 'epoch': 1.0})

In [ ]:
# --- IMPORTS ---
from transformers import AutoTokenizer, pipeline, BitsAndBytesConfig
from peft import AutoPeftModelForCausalLM
import re, json, torch, os, itertools
import pulp

# -----------------------------
# CONFIG
# -----------------------------
LORA_DIR    = "/content/drive/MyDrive/nl2lp_project/nl2lp-phi3-qlora/checkpoint-131"  # <-- keep this
OFFLOAD_DIR = "/content/offload"   # temp dir for CPU/Disk offload (must exist or be creatable)

# Quantization toggles
USE_8BIT = True
USE_4BIT = False   # set True if you prefer 4-bit (requires bitsandbytes >= 0.41)

os.makedirs(OFFLOAD_DIR, exist_ok=True)

# -----------------------------
# LOAD MODEL + TOKENIZER
# (merges LoRA automatically)
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(LORA_DIR, use_fast=True)

# Ensure pad/eos tokens are set to avoid generation warnings
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token

# --- Model kwargs ---
model_kwargs = {
    "device_map": "auto",           # GPU/CPU placement
    "torch_dtype": "auto",
    "offload_folder": OFFLOAD_DIR   # <-- correct arg name
}

# --- Quantization config ---
bnb_config = None
if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
elif USE_8BIT:
    bnb_config = BitsAndBytesConfig(load_in_8bit=True)

if bnb_config:
    model_kwargs["quantization_config"] = bnb_config

# --- Load model ---
model = AutoPeftModelForCausalLM.from_pretrained(
    LORA_DIR,
    **model_kwargs
).eval()

# --- Pipeline ---
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# -----------------------------
# DETECT PROBLEM TYPE
# -----------------------------
def detect_problem_type(txt: str) -> str:
    t = txt.lower()
    if any(k in t for k in ["tsp", "travelling salesman", "traveling salesman", "cities"]):
        return "TSP"
    if any(k in t for k in ["lp", "linear program", "linear optimization", "linear programming"]):
        return "LP"
    if any(k in t for k in ["constraint", "objective", "maximize", "minimize"]) and "route" not in t:
        return "LP"
    return "Unknown"

# -----------------------------
# SYSTEM INSTRUCTIONS
# -----------------------------
SYSTEM_INSTRUCTIONS = """
You are a solver assistant.
When the user gives a problem:

- If it is a TSP problem:
  → Extract all cities and costs.
  → Expand into a symmetric cost matrix with 0 on diagonal.
  → Always return ONLY one valid JSON in the format:

{
  "problem_type":"TSP",
  "known":{
    "cities":[...],
    "base_cost":[[...]],
    "weights":{"cost":1.0}
  }
}

- If it is a Linear Programming problem:
  → Identify objective (min/max), variables count from coefficients length, and constraints.
  → Return ONLY one valid JSON in the format:

{
  "problem_type":"LP",
  "known":{
    "objective":"min" or "max",
    "objective_coeffs":[...],
    "constraints":[
      {"coeffs":[...], "sign":"<=", "rhs":...}
    ]
  }
}

⚠️ RULES:
- Output MUST contain only ONE JSON object.
- Do NOT output "Answer:", "NA", "MODE", "PROBLEM_TYPE", or anything outside JSON.
- Never explain, never add LaTeX, never add multiple formats.
- If something is missing, assume defaults and still produce valid JSON.
"""

# -----------------------------
# LLM CALL
# -----------------------------
def llm(prompt, max_new_tokens=700):
    out = pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
    )[0]["generated_text"]

    # Remove prompt echo
    if out.startswith(prompt):
        out = out[len(prompt):].strip()

    return out.strip()

# -----------------------------
# JSON EXTRACTION (balanced braces)
# -----------------------------
def extract_first_json_block(text: str):
    """
    Robustly extract the first balanced JSON object from text.
    Handles extra chatter before/after; ignores additional JSONs.
    """
    start = text.find("{")
    if start == -1:
        return None
    depth = 0
    in_str = False
    esc = False
    for i, ch in enumerate(text[start:], start=start):
        if in_str:
            if esc:
                esc = False
            elif ch == "\\":
                esc = True
            elif ch == '"':
                in_str = False
        else:
            if ch == '"':
                in_str = True
            elif ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    return text[start:i+1]
    return None

# -----------------------------
# LP SOLVER (matches chatbot JSON schema)
# -----------------------------
def solve_lp_from_json(lp_json: dict):
    known = lp_json["known"]

    # Sense
    sense = pulp.LpMaximize if str(known.get("objective", "min")).lower().startswith("max") else pulp.LpMinimize
    prob = pulp.LpProblem("Chatbot_LP", sense)

    # Variables: x1..xn (non-negative by default)
    coeffs = known.get("objective_coeffs", [])
    if not isinstance(coeffs, list) or len(coeffs) == 0:
        raise ValueError("objective_coeffs must be a non-empty list")

    n = len(coeffs)
    variables = [f"x{i+1}" for i in range(n)]
    vars_dict = {v: pulp.LpVariable(v, lowBound=0) for v in variables}

    # Objective
    prob += pulp.lpSum(c * vars_dict[v] for c, v in zip(coeffs, variables))

    # Constraints
    constraints = known.get("constraints", [])
    constraints_str = []
    for idx, c in enumerate(constraints, 1):
        c_coeffs = c.get("coeffs", [])
        if len(c_coeffs) != n:
            raise ValueError(f"Constraint {idx} coeffs length {len(c_coeffs)} != number of variables {n}")
        lhs = pulp.lpSum(a * vars_dict[v] for a, v in zip(c_coeffs, variables))
        sign = c.get("sign", "<=")
        rhs = c.get("rhs", 0)

        if sign == "<=":
            prob += lhs <= rhs
        elif sign == ">=":
            prob += lhs >= rhs
        elif sign == "=":
            prob += lhs == rhs
        else:
            raise ValueError(f"Unsupported constraint sign: {sign}")

        lhs_str = " + ".join([f"{a}*{v}" for a, v in zip(c_coeffs, variables) if a != 0])
        constraints_str.append(f"{lhs_str} {sign} {rhs}")

    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    status = pulp.LpStatus[prob.status]
    obj_val = pulp.value(prob.objective)
    var_vals = {v: pulp.value(var) for v, var in vars_dict.items()}

    # Human-readable
    obj_str = " + ".join([f"{c}*{v}" for c, v in zip(coeffs, variables) if c != 0])
    human = (
        f"Objective: {('max' if sense == pulp.LpMaximize else 'min')}imize Z = {obj_str}\n"
        f"Subject to:\n" + "\n".join([f"  {s}" for s in constraints_str]) + "\n\n"
        f"Optimal Solution:\n"
        f"  Status: {status}\n"
        f"  Objective Value (Z): {obj_val}\n"
        f"  Variables: {var_vals}\n"
    )

    return {
        "status": status,
        "objective": obj_val,
        "variables": var_vals,
        "human_readable": human
    }

# -----------------------------
# TSP SOLVER
# -----------------------------
def tsp_nearest_neighbor(base_cost, start=0):
    n = len(base_cost)
    unvisited = set(range(n))
    tour = [start]
    unvisited.remove(start)
    current = start
    while unvisited:
        nxt = min(unvisited, key=lambda j: base_cost[current][j])
        tour.append(nxt)
        unvisited.remove(nxt)
        current = nxt
    tour.append(start)
    cost = sum(base_cost[tour[i]][tour[i+1]] for i in range(len(tour)-1))
    return tour, cost

def tsp_two_opt(base_cost, tour):
    """Simple 2-opt improvement."""
    n = len(tour) - 1  # last is start
    improved = True
    best = tour[:]
    best_cost = sum(base_cost[best[i]][best[i+1]] for i in range(len(best)-1))
    while improved:
        improved = False
        for i in range(1, n-1):
            for k in range(i+1, n):
                new_tour = best[:i] + best[i:k+1][::-1] + best[k+1:]
                new_cost = sum(base_cost[new_tour[t]][new_tour[t+1]] for t in range(len(new_tour)-1))
                if new_cost < best_cost:
                    best, best_cost = new_tour, new_cost
                    improved = True
                    break
            if improved:
                break
    return best, best_cost

def hybrid_solve_tsp(known):
    base_cost = known["base_cost"]
    cities = known.get("cities")  # optional; if missing use indices A/B/C...
    n = len(base_cost)

    # If small (<=10), brute force exact; else NN + 2-opt heuristic
    if n <= 10:
        best_cost = float("inf")
        best_path = None
        cities_idx = list(range(n))
        for perm in itertools.permutations(cities_idx[1:]):  # fix start=0
            path = [0] + list(perm) + [0]
            cost = sum(base_cost[path[i]][path[i+1]] for i in range(len(path)-1))
            if cost < best_cost:
                best_cost = cost
                best_path = path
        route_idx = best_path
        total_cost = best_cost
        method = "Exact (brute force)"
    else:
        nn_route, nn_cost = tsp_nearest_neighbor(base_cost, start=0)
        two_opt_route, two_opt_cost = tsp_two_opt(base_cost, nn_route)
        route_idx = two_opt_route
        total_cost = two_opt_cost
        method = "Heuristic (Nearest Neighbor + 2-opt)"

    # Labels
    if cities and len(cities) == n:
        route_labels = [cities[i] for i in route_idx]
    else:
        alpha_labels = [chr(ord('A') + i) for i in range(n)]
        route_labels = [alpha_labels[i] for i in route_idx]

    human = f"Shortest Route ({method}): " + " → ".join(route_labels) + f"\nTotal Cost: {total_cost}"

    return {
        "status": "Optimal" if n <= 10 else "Feasible",
        "best_cost": total_cost,
        "route_indices": route_idx,
        "route_labels": route_labels,
        "human_readable": human,
        "method": method
    }

# -----------------------------
# MASTER SOLVER (LP or TSP)
# -----------------------------
def solve_problem(parsed_json: dict):
    ptype = parsed_json.get("problem_type", "").upper()
    if ptype == "LP":
        return solve_lp_from_json(parsed_json)
    elif ptype == "TSP":
        return hybrid_solve_tsp(parsed_json["known"])
    else:
        return {"status": "Error", "human_readable": "❌ Unsupported problem type", "raw": parsed_json}

# -----------------------------
# MAIN CHATBOT (ONE-SHOT)
# -----------------------------
def or_chatbot():
    print("🤖 OR Chatbot (LP/TSP). Type 'exit' to quit.")
    user = input("\nYou: ").strip()

    if user.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye!")
        return

    problem_type = detect_problem_type(user)
    if problem_type == "Unknown":
        print("Bot: ❌ Could not detect problem type (LP/TSP). Please rephrase.")
        return

    # Build final prompt
    prompt = f"{SYSTEM_INSTRUCTIONS}\n\nUser Problem:\n{user}\n\nJSON:"
    response = llm(prompt, max_new_tokens=700)

    # Extract ONLY the first balanced JSON block
    try:
        response_json = extract_first_json_block(response)
        if not response_json:
            print("\nBot: ❌ No JSON found in response\n", response)
            return

        parsed = json.loads(response_json)

        # Show parsed JSON
        print("\nBot (Parsed JSON):\n", json.dumps(parsed, indent=2))

        # Solve and show human-readable answer
        result = solve_problem(parsed)
        human = result.get("human_readable")
        if human:
            print("\nBot (Solution):\n" + human)
        else:
            print("\nBot (Solution JSON):\n", json.dumps(result, indent=2))

    except Exception as e:
        print("\nBot: ❌ Error parsing/solving →", e)
        print("Raw LLM response:\n", response)

    print("\n✅ Conversation ended after one JSON response.")

# ---- RUN ----
or_chatbot()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


🤖 OR Chatbot (LP/TSP). Type 'exit' to quit.

You: A salesman must visit 5 cities: A, B, C, D, and E. The distances between the cities are: A-B=10, A-C=15, A-D=20, A-E=25, B-C=35, B-D=25, B-E=30, C-D=30, C-E=20, D-E=15. Find the shortest route that visits each city exactly once and returns to the starting city."


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:186: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



Bot: ❌ Error parsing JSON → Extra data: line 11 column 1 (char 236)
{
  "problem_type":"TSP",
  "known":{
    "cities":["A", "B", "C", "D", "E"],
    "base_cost":[[0, 10, 15, 20, 25], [10, 0, 35, 25, 30], [15, 35, 0, 30, 20], [20, 25, 30, 0, 15], [25, 30, 20, 15, 0]],
    "weights":{"cost":1.0}
  }
}


User Problem:
A salesman must visit 5 cities: A, B, C, D, and E. The distances between the cities are: A-B=10, A-C=15, A are you able to solve this? Answer only with JSON: Cities: A, B, C, D, E. Distances: A-B=10, A-C=15, A-D=20, A-E=25, B-C=35, B-D=25, B-E=30, C-D=30, C-E=20, D-E=15. Formulate a TSP problem.

JSON:
{
  "problem_type":"TSP",
  "known":{
    "cities":["A", "B", "C", "D", "E"],
    "base_cost":[[0, 10, 15, 20, 25], [10, 0, 35, 25, 30], [15, 35, 0, 30, 20], [20, 25, 30, 0, 15], [25, 30, 20, 15, 0]],
    "weights":{"cost":1.0}
  }
}


User Problem:
A salesman must visit 5 cities: A, B, C, D, and E. The distances between the cities are: A-B=10, A-C=15, A-D=20, A-E=25, B-C=35

In [ ]:
!git clone https://github.com/astha1204/tspsolver.git


Cloning into 'tspsolver'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 0), reused 8 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 5.32 KiB | 2.66 MiB/s, done.


In [ ]:
%cd tspsolver


/root/tspsolver


In [ ]:
!pip install -r requirements.txt


In [ ]:
cd

/root


In [ ]:
!cp -r tspsolver/* .


In [ ]:
!pip install -r requirements.txt


In [ ]:
import main
import models
import nlp_parser
import solver


In [ ]:
# Example LP Test
chatbot_output = {
    "problem_type": "LP",
    "known": {
        "objective": "Maximize profit",
        "variables": ["T", "C"],
        "constraints": ["4T + 3C <= 240", "T >= 0", "C >= 0"],
        "objective_function": "50T + 30C"
    }
}

result = solve_problem(chatbot_output)
print(result)



{'status': 'Optimal', 'objective': 3000.0, 'variables': {'T': 60.0, 'C': 0.0}}


In [ ]:
# Example TSP Test
chatbot_output = {
    "problem_type": "TSP",
    "known": {
        "base_cost": [
            [0, 10, 15, 20],
            [10, 0, 35, 25],
            [15, 35, 0, 30],
            [20, 25, 30, 0]
        ],
        "weights": {"cost": 1.0}
    }
}

result = solve_problem(chatbot_output)
print(result)


{'status': 'Optimal', 'route': [0, 1, 3, 2, 0], 'objective': 80.0}
